In [168]:
import sys
import json
aggregation = {'distance':1000}
constant = {'cst_incline':1}
road_weight = {'residential':0.5,
            'secondary_link':0.3,
            'secondary':0.3,
            'tertiary':0.3,
            'cycleway':1.1,
            'primary':0.3,
            'motorway_link':0,
            'primary_link':0.3,
            'tertiary_link':0.3,
            'motorway':0,
            'trunk':0.2, 
            'trunk_link':0.2  
            }
shared_cycleway_weight = {'residential':0.8,
            'secondary_link':0.5,
            'secondary':0.5,
            'tertiary':0.5,
            'cycleway':1.1,
            'primary':0.5,
            'motorway_link':0,
            'primary_link':0.5,
            'tertiary_link':0.5,
            'motorway':0,
            'trunk':0.4, 
            'trunk_link':0.4   
        }

params = {'aggregation':aggregation,'constant':constant, 'road_weight': road_weight, 'shared_cycleway_weight': shared_cycleway_weight}

         

default = {'training_folder': '../../scenarios/quebec', 'params':params} # Default execution parameters
manual, argv = (True, default) if 'ipykernel' in sys.argv[0] else (False, dict(default, **json.loads(sys.argv[1])))
print(argv)


{'training_folder': '../../scenarios/quebec', 'params': {'aggregation': {'distance': 1000}, 'constant': {'cst_incline': 1}, 'road_weight': {'residential': 0.5, 'secondary_link': 0.3, 'secondary': 0.3, 'tertiary': 0.3, 'cycleway': 1.1, 'primary': 0.3, 'motorway_link': 0, 'primary_link': 0.3, 'tertiary_link': 0.3, 'motorway': 0, 'trunk': 0.2, 'trunk_link': 0.2}, 'shared_cycleway_weight': {'residential': 0.8, 'secondary_link': 0.5, 'secondary': 0.5, 'tertiary': 0.5, 'cycleway': 1.1, 'primary': 0.5, 'motorway_link': 0, 'primary_link': 0.5, 'tertiary_link': 0.5, 'motorway': 0, 'trunk': 0.4, 'trunk_link': 0.4}}}


In [169]:
import os
import geopandas as gpd
import pandas as pd
sys.path.insert(0, r'../../../quetzal') # Add path to quetzal
from quetzal.engine.road_model import RoadModel
from quetzal.engine.pathfinder_utils import get_path, parallel_dijkstra, build_index, sparse_matrix
from quetzal.engine.msa_utils import get_zone_index, assign_volume
import numpy as np
import random
from shapely.geometry import Point
from typing import Tuple
num_cores = 1
#from sklearn.cluster import KMeans
from syspy.spatial.spatial import nearest, agglomerative_clustering, voronoi_diagram_dataframes
from quetzal.engine.pathfinder_utils import simple_routing,get_path

In [170]:
def drop_duplicated_links(links: gpd.GeoDataFrame,
                          sort_column: str = 'maxspeed', 
                          ascending: bool = False, 
                          return_dropped:bool = False)-> Tuple[gpd.GeoDataFrame,list]:
    '''
    drop duplicated links (a,b) with condition sort_column. if maxspeed and ascending=False, keep faster road
    '''
    before = set(links.index)
    links['dup'] = links['a'] + links['b']
    links = links.sort_values(sort_column, ascending=ascending).drop_duplicates('dup').sort_index()
    links = links.drop(columns='dup')
    after = set(links.index)
    diff = list(before-after)
    print(len(diff), 'links dropped')
    return links, diff

def get_epsg(lat: float, lon: float) -> int:
    '''
    return EPSG in meter for a given (lat,lon)
    lat is north south 
    lon is est west
    '''
    return int(32700 - round((45 + lat) / 90, 0) * 100 + round((183 + lon) / 6, 0))

def zones_nearest_node(zones,nodes,drop_duplicates=False):
    # getting zones centroids
    centroid = zones.copy()
    centroid['geometry'] = centroid.centroid
    # finding nearest node
    neigh = nearest(centroid, nodes, n_neighbors=1).rename(columns={'ix_one': 'zone_index', 'ix_many': 'node_index'})
    zone_node_dict = neigh.set_index('zone_index')['node_index'].to_dict()
    centroid['node_index'] = centroid.index.map(zone_node_dict.get)
    print('max_distance found: ', neigh['distance'].max())
    # check for duplicated nodes. if there is. drop the duplicated zones.
    if drop_duplicates:
        if len(centroid.drop_duplicates('node_index')) != len(centroid):
            print('there is zones associates to the same road_node')
            # duplicated = centroid[centroid['node_index'].duplicated()]['node_index'].values
            print('dropping zones: ')
            print(centroid[centroid['node_index'].duplicated()].index.values)
            centroid = centroid.drop_duplicates('node_index')
    return centroid

In [171]:
base_folder = argv['training_folder']
road_folder = base_folder + '/inputs/road/'
demand_folder = base_folder +'/inputs/'
od_folder = base_folder + '/inputs/od/'
output_folder = base_folder +'/outputs/'
print(road_folder)
if not os.path.exists(output_folder):
    os.makedirs(output_folder)


../../scenarios/quebec/inputs/road/


# inputs

In [172]:
cst_incline = argv['params' ]['constant']['cst_incline']
cst_road = argv['params']['road_weight']
cst_shared = argv['params']['shared_cycleway_weight']

In [173]:
#make sure its float
cst_incline = float(cst_incline)
cst_road = {k:float(v) for k,v in cst_road.items()}
cst_shared = {k:float(v) for k,v in cst_shared.items()}

In [174]:
links = gpd.read_file(road_folder + 'road_links.geojson') 
nodes = gpd.read_file(road_folder + 'road_nodes.geojson')
links = links.set_index('index')
nodes = nodes.set_index('index')

In [175]:
demand = pd.read_csv(demand_folder+'demand.csv')
assert all([col in demand.columns for col in ['lon_ori','lat_ori','lon_des','lat_des','volume'] ]), 'need origin,destination and volume in demand'
demand = demand[['lon_ori','lat_ori','lon_des','lat_des','volume']]

In [179]:
od_test = gpd.read_file(od_folder + 'od.geojson')

In [182]:
if 'name' not in od_test.columns:
    od_test['name'] = od_test['index']

# agg demand

format demand and create zone

In [62]:
demand['origin'] = [*zip(demand['lon_ori'],demand['lat_ori'])]
demand['destination'] = [*zip(demand['lon_des'],demand['lat_des'])]
demand = demand.drop(columns=['lat_ori','lon_ori','lat_des','lon_des'])

In [63]:
# get unique zones
zones_set = set(demand['origin'].values).union(set(demand['destination'].values))
zones_dict = {val:'zone_'+str(i) for i,val in enumerate(zones_set)}
zones_df = [{'index':zone,'geometry':Point(val)} for val,zone in zones_dict.items()]

In [64]:
demand['origin'] = demand['origin'].apply(lambda x: zones_dict.get(x))
demand['destination'] = demand['destination'].apply(lambda x: zones_dict.get(x))
zones = gpd.GeoDataFrame(zones_df,crs=4326).set_index('index')

Clustering

In [65]:
agg_dist = float(argv['params']['aggregation']['distance'])
epsg = get_epsg(zones.iloc[0]['geometry'].y, zones.iloc[0]['geometry'].x)
label = agglomerative_clustering(zones.to_crs(epsg), distance_threshold = agg_dist)
zones['cluster'] = label

In [66]:
print(len(zones),'zones agg to',len(zones['cluster'].unique()),' zones')

20000 zones agg to 626  zones


In [67]:
# create a dict zone:cluster
zones['cluster'] = 'zone_' + zones['cluster'].astype(str)
cluster_dict = zones['cluster'].to_dict()

In [68]:
# drop duplicated cluster and rename index with new cluster as zones_id
zones = zones.drop_duplicates('cluster')
zones = zones.set_index('cluster')
zones.index.name='index'

 AGG demand on new zones (volime is sum)

In [69]:
# apply new cluster fict to the demand
demand['origin'] = demand['origin'].apply(lambda x: cluster_dict.get(x))
demand['destination'] = demand['destination'].apply(lambda x: cluster_dict.get(x))

In [70]:
demand = demand.groupby(['origin','destination']).agg(sum).reset_index()
demand.index.name = 'index'

# export zones and demand in outputs

In [71]:
#zones.to_file(output_folder + 'centroids.geojson',driver='GeoJSON')

In [72]:
# Group the DataFrame by "origin" and "destination", and then convert to JSON
grouped = demand.groupby(["origin", "destination"])["volume"].sum().reset_index()
json_data = {}
for _, row in grouped.iterrows():
    origin = row["origin"]
    destination = row["destination"]
    volume = row["volume"]
    if origin not in json_data:
        json_data[origin] = {}
    json_data[origin][destination] = volume

    
with open(output_folder + 'zones.json', 'w') as json_file:
    json.dump({'volume':json_data},json_file)

In [73]:
convex_hull = zones.unary_union.convex_hull.buffer(1e-3)
voronoi = voronoi_diagram_dataframes(zones['geometry'])
voronoi = gpd.GeoDataFrame(voronoi[0],crs=4326)
voronoi = voronoi.clip(convex_hull)
voronoi.to_file(output_folder + 'zones.geojson',driver='GeoJSON')

/home/boivin/.pyenv/versions/3.8.6/envs/quetzal_env/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:127: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [74]:
'''
def create_zones_from_nodes(nodes,num_zones=200):
    nodes['x'] = nodes['geometry'].apply(lambda p:p.x)
    nodes['y'] = nodes['geometry'].apply(lambda p:p.y)
    cluster = KMeans(n_clusters=num_zones,random_state=0,n_init='auto')
    cluster.fit(nodes[['x','y']].values)
    geom = [Point(val) for val in cluster.cluster_centers_]
    zones = gpd.GeoDataFrame(range(len(geom)),geometry=geom,crs=4326).drop(columns=0)
    zones.index = 'zone_' + zones.index.astype(str)
    return zones
#create_zones_from_nodes(zones).plot()
'''

"\ndef create_zones_from_nodes(nodes,num_zones=200):\n    nodes['x'] = nodes['geometry'].apply(lambda p:p.x)\n    nodes['y'] = nodes['geometry'].apply(lambda p:p.y)\n    cluster = KMeans(n_clusters=num_zones,random_state=0,n_init='auto')\n    cluster.fit(nodes[['x','y']].values)\n    geom = [Point(val) for val in cluster.cluster_centers_]\n    zones = gpd.GeoDataFrame(range(len(geom)),geometry=geom,crs=4326).drop(columns=0)\n    zones.index = 'zone_' + zones.index.astype(str)\n    return zones\n#create_zones_from_nodes(zones).plot()\n"

# split oneway quenedi links

In [75]:
links

,highway,speed,lanes,name,oneway,surface,cycleway,a,b,cycleway_reverse,...,time,incline,route_width,route_color,speed_r,lanes_r,name_r,surface_r,time_r,geometry
index,,,,,,,,,,,,,,,,,,,,,
road_link_0,residential,38.772474,1.0,9e Rue,1,asphalt,no,road_node_1128,road_node_1710,no,...,13.576979,-0.783613,2,808080,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-71.23089 46.82516, -71.23042 46.8..."
road_link_1,residential,38.772474,1.0,Rue Sutherland,1,asphalt,no,road_node_4797,road_node_5141,no,...,4.214099,5.036586,2,808080,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-71.22443 46.80920, -71.22403 46.8..."
road_link_2,residential,38.772474,2.0,Rue Dina-Bélanger,0,asphalt,no,road_node_4509,road_node_8592,no,...,7.262803,-2.196372,2,808080,38.772474,2.0,Rue Dina-Bélanger,asphalt,7.262803,"LINESTRING (-71.25063 46.78553, -71.25045 46.7..."
road_link_3,residential,30.000000,2.0,Rue De Maisonneuve,0,asphalt,shared,road_node_5919,road_node_5519,shared,...,18.416490,1.866008,2,808080,30.000000,2.0,Rue De Maisonneuve,asphalt,18.416490,"LINESTRING (-71.22494 46.80448, -71.22369 46.8..."
road_link_4,primary_link,50.000000,1.0,NaN,1,asphalt,no,road_node_5074,road_node_2923,no,...,6.307702,0.000000,4,FCD6A4,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-71.25305 46.81402, -71.25282 46.8..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
road_link_12493,trunk,50.000000,2.0,Boulevard Robert-Bourassa,1,NaN,no,road_node_6061,road_node_7264,no,...,48.208189,2.223768,4,E892A2,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-71.33864 46.85090, -71.33893 46.8..."
road_link_12494,trunk,50.000000,3.0,Autoroute Robert-Bourassa,1,NaN,no,road_node_6845,road_node_8545,no,...,22.165195,0.930498,4,E892A2,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-71.27538 46.77401, -71.27548 46.7..."
road_link_12495,trunk,50.000000,3.0,Autoroute Robert-Bourassa,1,NaN,no,road_node_3197,road_node_9712,no,...,22.086820,-0.747063,4,E892A2,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-71.27802 46.77610, -71.27766 46.7..."


In [76]:
# split quenedi road links
self = RoadModel(links,nodes,zones,ff_time_col='time')

In [77]:

print('split rlinks to oneways')
self.split_quenedi_rlinks()
#self.zones_nearest_node()

split rlinks to oneways


In [78]:
links = self.road_links
del self

# tag cycleways

In [79]:

index = [idx for idx in links.index if not idx.endswith('_r')]
index_r = [idx for idx in links.index if idx.endswith('_r')]
assert len(index)+len(index_r) == len(links)

links['isroad'] = True
ls = links.loc[index].loc[links['cycleway'].isin(['yes','shared'])].index
links.loc[ls,'isroad'] = False
ls = links.loc[index_r].loc[links['cycleway_reverse'].isin(['yes','shared'])].index
links.loc[ls,'isroad'] = False


# inclines

In [80]:
links['incline'] = links['incline'].astype(float)
links['incline'] = links['incline'].fillna(0)


In [81]:
# reverse incline on reversed links
links.loc[index_r,'incline'] = -links.loc[index_r,'incline']
#create this column for vizualisation
links['incline (abs)'] = abs(links['incline'])

# apply Weight

In [82]:
# incline weigth (starting point)
links['eff_speed'] = 20  * (1 - cst_incline * np.sin(np.deg2rad(links['incline'])))


In [83]:
# derate it with road weight
links['cst_road'] = links['highway'].apply(lambda x: cst_road.get(x,1))
links.loc[links['isroad'],'eff_speed'] = links.loc[links['isroad'],'eff_speed'] * links.loc[links['isroad'],'cst_road']

In [84]:
# derate it with shared cycleway weight
links['cst_shared'] = links['highway'].apply(lambda x: cst_shared.get(x,1))

ls = links.loc[index].loc[links['cycleway']=='shared'].index
links.loc[ls,'eff_speed'] = links.loc[ls,'eff_speed'] * links.loc[ls,'cst_shared']

ls = links.loc[index_r].loc[links['cycleway_reverse']=='shared'].index
links.loc[ls,'eff_speed'] = links.loc[ls,'eff_speed'] * links.loc[ls,'cst_shared']

transform effective speed to a weight

In [85]:
# anything with neg speed round to 0. (dijkstra need positive weight)
links['eff_speed'] = links['eff_speed'].apply(lambda x : max(x,0))

In [86]:
links['weight'] = links['length']/(links['eff_speed']*1000/3600)

In [87]:
links = links.drop(columns=['cst_road','cst_shared'])

# zone to nodes

get the list of nodes with weight != inf. we do not want to route from or to nodes that are not cycle (ex motorway)

In [88]:
tlinks = links[np.isfinite(links['weight'])][['a','b']]
nodes_set = set(tlinks['a']).union(set(tlinks['b']))
possible_nodes = nodes.loc[list(nodes_set)].sort_index()
del tlinks,nodes_set

In [89]:
zones = zones_nearest_node(zones, possible_nodes)

/tmp/ipykernel_184022/3645094334.py:28: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroid['geometry'] = centroid.centroid


max_distance found:  0.022534254147859218


# Shortest path + volume assignment

In [90]:
zones_nodes_dict = zones['node_index'].to_dict()
demand['o_zone'] = demand['origin']
demand['d_zone'] = demand['destination']
demand['origin'] = demand['origin'].apply(lambda x: zones_nodes_dict.get(x))
demand['destination'] = demand['destination'].apply(lambda x: zones_nodes_dict.get(x))

In [91]:
df = links[['a', 'b','weight']].copy()

In [92]:
df, dropped = drop_duplicated_links(df,sort_column='weight',ascending=False)

316 links dropped


In [93]:
edges = df[['a', 'b', 'weight']].values  # to build the index once and for all
index = build_index(edges)
reversed_index = {v: k for k, v in index.items()}
# apply sparse index on zones
demand, zones_indices = get_zone_index(df, demand, index)

In [94]:
# apply sparse index on links
df['sparse_a'] = df['a'].apply(lambda x: index.get(x))
df['sparse_b'] = df['b'].apply(lambda x: index.get(x))
volumes_sparse_keys = list(zip(df['sparse_a'],df['sparse_b']))

odv = demand[['o', 'd', 'volume']].values

In [95]:
edges = df[['a', 'b', 'weight']].values
sparse, _ = sparse_matrix(edges, index=index)
time_matrix, predecessors = parallel_dijkstra(sparse,
                                              directed=True,
                                              indices=zones_indices,
                                              return_predecessors=True,
                                              num_core=num_cores,
                                              keep_running=True)

# this give OD_time/time_matrix on each links. then X links time for the ratio links_time/tot_time


In [96]:
ab_volumes = assign_volume(odv,predecessors,volumes_sparse_keys,reversed_index)

restrict links to df.index (dropped duplicated links). if not. volume will not be assign on the right link (when duplicated) or, maybe on both.

In [97]:
links['volume'] = 0
links.loc[df.index,'volume'] = links.loc[df.index].set_index(['a', 'b']).index.map(ab_volumes.get)

# cleaning and exporting

In [98]:
col_r = [col for col in links.columns if col.endswith('_r')]
links = links.drop(columns=col_r)
links = links.drop(columns=['isroad'])


In [99]:
links.to_file(output_folder + 'loaded_links.geojson',driver='GeoJSON')

In [100]:
links['volume'].sum()

25486677

# OD test

In [119]:
od_test['geometry_o'] = od_test['geometry'].apply(lambda g: Point(g.coords[:][0]))
od_test['geometry_d'] = od_test['geometry'].apply(lambda g: Point(g.coords[:][1]))

In [120]:
# find nearest node with KNN. nodes are now the origin and destination.
od_test['geometry'] = od_test['geometry_o']
neigh = nearest(od_test, possible_nodes, n_neighbors=1).rename(columns={'ix_one': 'zone_index', 'ix_many': 'node_index'})
zone_node_dict = neigh.set_index('zone_index')['node_index'].to_dict()
od_test['origin'] = od_test.index.map(zone_node_dict.get)

od_test['geometry'] = od_test['geometry_d']
neigh = nearest(od_test, possible_nodes, n_neighbors=1).rename(columns={'ix_one': 'zone_index', 'ix_many': 'node_index'})
zone_node_dict = neigh.set_index('zone_index')['node_index'].to_dict()
od_test['destination'] = od_test.index.map(zone_node_dict.get)

od_test = od_test.drop(columns=['geometry_o','geometry_d','geometry'])

In [121]:
o_nodes = od_test['origin'].values
d_nodes = od_test['destination'].values

time_mat, predecessors, node_index = simple_routing(o_nodes,d_nodes,links,weight_col='weight',return_predecessors=True)
reversed_index = {v: k for k, v in node_index.items()}

routes = gpd.GeoDataFrame()
for o,d in  enumerate(od_test['destination'].values):
    path = get_path(predecessors, o, node_index[d])
    path = list(zip(path[:-1], path[1:]))

    path = [(reversed_index[k[0]], reversed_index[k[1]]) for k in path]
    links_dict = links.reset_index().set_index(['a','b'])['index'].to_dict()
    path = [*map(links_dict.get,path)]

    route = links.loc[path]
    route['od_name'] =  od_test.iloc[o]['name']
    routes = pd.concat([routes,route])

In [122]:
routes.to_file(output_folder + 'od_routes.geojson',driver='GeoJSON')

# end

# merging

In [107]:
'''
def merge_quenedi_rlinks(road_links,new_col):
    if 'oneway' not in road_links.columns:
        print('no column oneway. do not merge')
        return
    #get reversed links
    index_r = [idx for idx in road_links.index if idx.endswith('_r')]
    if len(index_r) == 0:
        print('all oneway, nothing to merge')
        return
    links_r = road_links.loc[index_r].copy()
    # create new reversed column with here speed and time
    links_r[new_col + '_r'] = links_r[new_col]
    # reindex with initial non _r index to merge
    links_r.index = links_r.index.map(lambda x: x[:-2])
    links_r = links_r[[new_col + '_r']]
    # drop added _r links, merge new here columns to inital two way links.
    road_links = road_links.drop(index_r, axis=0)
    # drop column if they exist before merge. dont want duplicates
    if new_col + '_r' in road_links.columns:
        road_links = road_links.drop(columns=new_col + '_r')
    road_links = pd.merge(road_links, links_r, left_index=True, right_index
                                =True, how='left')
    return road_links
'''

"\ndef merge_quenedi_rlinks(road_links,new_col):\n    if 'oneway' not in road_links.columns:\n        print('no column oneway. do not merge')\n        return\n    #get reversed links\n    index_r = [idx for idx in road_links.index if idx.endswith('_r')]\n    if len(index_r) == 0:\n        print('all oneway, nothing to merge')\n        return\n    links_r = road_links.loc[index_r].copy()\n    # create new reversed column with here speed and time\n    links_r[new_col + '_r'] = links_r[new_col]\n    # reindex with initial non _r index to merge\n    links_r.index = links_r.index.map(lambda x: x[:-2])\n    links_r = links_r[[new_col + '_r']]\n    # drop added _r links, merge new here columns to inital two way links.\n    road_links = road_links.drop(index_r, axis=0)\n    # drop column if they exist before merge. dont want duplicates\n    if new_col + '_r' in road_links.columns:\n        road_links = road_links.drop(columns=new_col + '_r')\n    road_links = pd.merge(road_links, links_r, left

In [108]:
#links = merge_quenedi_rlinks(links,'volume')

# test